In [3]:
import pandas as pd
import imp
import pprint
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.font_manager import FontProperties
from tqdm.notebook import tqdm
%matplotlib qt
fontP = FontProperties()
fontP.set_size('xx-small')

In [25]:
import eval
imp.reload(eval)



settings = {
        'type': "nhl_SOG",
        'csvs': ["my_g", "my_o"],
        'sep': ';',
        'bet_on': ['under', 'over'],
        'kelly_risk': 0.1,
        'start': 1,
        'verb': True,
        'kelly_up_lim': 0.4,
        'kelly_low_lim': 0.0,
        'proba_up_lim': 0.7,
        'proba_low_lim': 0.5,
        'rev_data': False
}
e = eval.daily(settings)
ret = e.eval()

for key, value in ret.items():
        csv = value['settings']['csvs'][0]
        kelly_risk = value['settings']['kelly_risk']

        # Plot
        fig = plt.figure()
        plt.title(f'ROI for \"{csv}.csv\" with risk of: {kelly_risk}')
        plt.xlabel('Potential bets (over and under for each player in each game)')
        plt.ylabel('ROI (if start = 1 else just return)')
        plt.grid()
        plots=[]

        # Plot each day
        plots.append(plt.plot(ret[f'{csv}-{kelly_risk}']['data']['days'], ret[f'{csv}-{kelly_risk}']['data']['days_return'], label = f"{csv}-days")[0])

        # Plot each bet
        x = list(range(0, ret[f'{csv}-{kelly_risk}']['total_bets']))
        plots.append(plt.plot(x, ret[f'{csv}-{kelly_risk}']['data']['bets_return'], label = f"{csv}-bets")[0])

        plt.legend(handles=plots, title='Lines', bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
        plt.show()

Csv: "my_g.csv" and Pre: 0.1
Ending 2021-04-27 with 103.57% of starting capital
Ending 2021-04-28 with 106.56% of starting capital
Ending 2021-04-29 with 103.45% of starting capital
Ending 2021-05-02 with 102.07% of starting capital
Ending 2021-05-03 with 108.56% of starting capital
Ending 2021-05-04 with 119.91% of starting capital
Ending 2021-05-05 with 122.61% of starting capital
Ending 2021-05-06 with 134.15% of starting capital
Ending 2021-05-07 with 146.58% of starting capital
Ending 2021-05-09 with 143.89% of starting capital
Ending 2021-05-12 with 137.65% of starting capital
Ending 2021-05-17 with 135.19% of starting capital
Ending 2021-05-18 with 146.95% of starting capital
Ending 2021-05-19 with 127.04% of starting capital
Ending 2021-05-20 with 122.36% of starting capital
Ending 2021-05-21 with 128.28% of starting capital
Ending 2021-05-22 with 127.55% of starting capital
Ending 2021-05-23 with 117.56% of starting capital
Ending 2021-05-24 with 104.04% of starting capital
be

In [68]:
import eval
imp.reload(eval)


settings = {
        'type': "CL_HomeXAway",
        'csvs': ["CL"],
        'bet_on': ["home", "draw", "away"],#, "home_draw", "home_away", "home_away"],
        'kelly_risk': 0.2,
        'start': 1,
        'verb': True,
        'kelly_up_lim': 1,
        'kelly_low_lim': 0,
        'proba_up_lim': 1,
        'proba_low_lim': 0,
        'rev_data': False
}
e = eval.daily(settings)
ret = e.eval()

for key, value in ret.items():
        csv = value['settings']['csvs'][0]
        kelly_risk = value['settings']['kelly_risk']

        # Plot
        fig = plt.figure()
        plt.title(f'ROI for \"{csv}.csv\" with risk of: {kelly_risk}')
        plt.xlabel('Potential bets (over and under for each player in each game)')
        plt.ylabel('ROI (if start = 1 else just return)')
        plt.grid()
        plots=[]

        # Plot each day
        plots.append(plt.plot(ret[f'{csv}-{kelly_risk}']['data']['days'], ret[f'{csv}-{kelly_risk}']['data']['days_return'], label = f"{csv}-days")[0])

        # Plot each bet
        x = list(range(0, ret[f'{csv}-{kelly_risk}']['total_bets']))
        plots.append(plt.plot(x, ret[f'{csv}-{kelly_risk}']['data']['bets_return'], label = f"{csv}-bets")[0])

        plt.legend(handles=plots, title='Lines', bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
        plt.show()

Csv: "CL.csv" and Pre: 0.2
Ending 4560.0 with 141.32% of starting capital
Ending 4561.0 with 136.19% of starting capital
Ending 4562.0 with 130.37% of starting capital
Ending 4564.0 with 127.43% of starting capital
Ending 4565.0 with 119.16% of starting capital
Ending 4566.0 with 200.25% of starting capital
Ending 4567.0 with 186.27% of starting capital
Ending 4568.0 with 179.6% of starting capital
Ending 4569.0 with 195.25% of starting capital
Ending 4570.0 with 198.04% of starting capital
Ending 4571.0 with 183.54% of starting capital
Ending 4572.0 with 159.73% of starting capital
Ending 4573.0 with 171.61% of starting capital
Ending 4574.0 with 211.49% of starting capital
Ending 4575.0 with 187.46% of starting capital
Ending 4576.0 with 174.45% of starting capital
Ending 4577.0 with 220.17% of starting capital
Ending 4579.0 with 197.49% of starting capital
Ending 4580.0 with 274.34% of starting capital
Ending 4581.0 with 265.61% of starting capital
Ending 4582.0 with 227.75% of star